In [19]:
import tensorflow as tf
import numpy as np


In [20]:
!nvidia-smi

Fri Jun 25 17:59:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   45C    P8     9W / 198W |   7924MiB /  8116MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
model = tf.keras.applications.ResNet50(include_top = False, input_shape = (64, 64, 3))
print(model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [22]:
feats = []
for layer in model.layers:
    if(layer.name[4] == "2"):
        layer1 = layer.output
    if(layer.name[4] == "3"):
        layer2 = layer.output
    if(layer.name[4] == "4"):
        layer3 = layer.output
    if(layer.name[4] == "5"):
        layer4 = layer.output

feats = [layer1, layer2, layer3, layer4]

In [23]:
print(feats)

[<KerasTensor: shape=(None, 16, 16, 256) dtype=float32 (created by layer 'conv2_block3_out')>, <KerasTensor: shape=(None, 8, 8, 512) dtype=float32 (created by layer 'conv3_block4_out')>, <KerasTensor: shape=(None, 4, 4, 1024) dtype=float32 (created by layer 'conv4_block6_out')>, <KerasTensor: shape=(None, 2, 2, 2048) dtype=float32 (created by layer 'conv5_block3_out')>]


In [24]:
activation_model = tf.keras.models.Model(inputs=model.input, outputs=feats)
print(activation_model)

In [25]:
activation_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [26]:
y_train = np.load("/home/notchla/Documents/mias_y_train.npy", allow_pickle=True)

In [27]:
PATCHING_SIZE = 64
ksize_rows = PATCHING_SIZE
ksize_cols = PATCHING_SIZE
strides_rows = PATCHING_SIZE
strides_cols = PATCHING_SIZE
overlap = 8

ksizes = [1, ksize_rows, ksize_cols, 1]
strides = [1, strides_rows-overlap, strides_cols-overlap, 1]
rates = [1, 1, 1, 1]
padding = "VALID"

y_train_patches = tf.image.extract_patches(images = y_train, sizes = ksizes, strides = strides, rates=rates, padding=padding)

In [28]:
axis_0, axis_1, axis_2, _ = y_train_patches.shape
print(axis_0, axis_1, axis_2)
print(y_train_patches.shape)

258 9 9
(258, 9, 9, 4096)


In [29]:
IMAGES_TRAIN = axis_0

In [30]:
nsamples_train = axis_0*axis_1*axis_2
y_train_patches = np.reshape(y_train_patches, [nsamples_train, PATCHING_SIZE, PATCHING_SIZE, 1])

In [31]:
del y_train

In [32]:
y_train_perceptual = np.repeat(y_train_patches, 3, -1)
#y_val_perceptual = np.repeat(y_val_patches, 3, -1)

In [33]:
y_train_perceptual = activation_model.predict(y_train_perceptual[:4000], batch_size=4, verbose = 1)

2000/2000 [==============================] - 16s 8ms/step


ResourceExhaustedError: OOM when allocating tensor with shape[8000,8,8,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat

In [17]:
!nvidia-smi

Fri Jun 25 17:58:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   48C    P2    53W / 198W |   7924MiB /  8116MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          15942        8395        6095         121        1451        7131
Swap:          2047         582        1465


In [16]:
y_stage1 = np.asarray(y[0], dtype=object)
y_stage2 = np.asarray(y[1], dtype=object)
y_stage3 = np.asarray(y[2], dtype=object)
y_stage4 = np.asarray(y[3], dtype=object)

NameError: name 'y' is not defined

In [ ]:
y_perceptual = []
for i in range(y_stage4.shape[0]):
    temp = []
    temp.append(y_stage1[i])
    temp.append(y_stage2[i])
    temp.append(y_stage3[i])
    temp.append(y_stage4[i])
    y_perceptual.append(temp)